#Init

In [ ]:
!pip install sklearn_som

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy import stats
from sklearn.neighbors import KNeighborsClassifier
import statsmodels.api as sm
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, plot_confusion_matrix
from statsmodels.stats.stattools import durbin_watson
from scipy.stats import shapiro
from sklearn_som.som import SOM

In [ ]:
index = 291951

In [ ]:
wina = pd.read_csv('https://gist.githubusercontent.com/sikor272/69f708f3d2d0616b767666a8b7820e02/raw/b13bc08eded544893cdcc9ded0b4599a81578ee1/winequality-red.csv', sep=";")

In [ ]:
columns_train = ['Stała kwasowość', 'Lotna kwasowość', 'Kwas cytrynowy', 'Cukier resztkowy',
                'Chlorki', 'Wolny dwutlenek siarki', 'Całkowity dwutlenek siarki', 'Gęstość',
                'pH', 'Siarczany', 'Alkohol']
columns_result = ['Jakość']
wina.columns = columns_train + columns_result

#Podział

In [ ]:
x_train, x_test , y_train, y_test = train_test_split(wina[columns_train], wina[columns_result], train_size=0.7, random_state=index, stratify=wina[columns_result])
wina_train = pd.concat([x_train, y_train], axis=1)
wina_test = pd.concat([x_test, y_test], axis=1)

y_train_array = np.array(y_train).reshape((-1, 1))
y_test_array = np.array(y_test).reshape((-1, 1))

In [ ]:
print(wina.shape)

print(x_train.shape)
print(y_train.shape)

print(x_test.shape)
print(y_test.shape)

In [ ]:
columns_train_no_corr = ['Lotna kwasowość', 'Cukier resztkowy',
                'Chlorki', 'Całkowity dwutlenek siarki', 'Siarczany', 'Alkohol']
x_train_no_corr, x_test_no_corr , y_train_no_corr, y_test_no_corr = train_test_split(wina[columns_train_no_corr], wina[columns_result], train_size=0.7, random_state=index, stratify=wina[columns_result])

#Analiza Danych

In [ ]:
wina.isna().sum()

In [ ]:
wina.duplicated().sum()

In [ ]:
wina.groupby(columns_result).size().reset_index(name='Ilość')

In [ ]:
plt.rcParams['figure.figsize'] = [10, 10]
wina_train.hist()
plt.show()

In [ ]:
wina_train.describe().rename(index={'count' : 'Ilość', 'mean' : 'Średnia', 'min': 'Minimum', 'max' : 'Maksimum', 'std': 'Odchylenie standardowe'}).applymap('{:.3f}'.format)

In [ ]:
wina_train.groupby(columns_result).size().reset_index(name='Ilość')

In [ ]:
wina_train.groupby(columns_result).mean().reset_index().T

##Korelacja
Zbór treningowy

In [ ]:
corr = wina_train.corr()
plt.rcParams['figure.figsize'] = [10, 10]
mask = np.triu(np.ones_like(corr, dtype = bool)).T
sns.heatmap(corr, mask = mask, annot=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=wina_train.columns, xticklabels=wina_train.columns)
plt.show()

#Klasyfikacja -> KNN

In [ ]:
standar_x = StandardScaler()
standar_x.fit(x_train_no_corr)
x_train_std = standar_x.transform(x_train_no_corr)
x_test_std = standar_x.transform(x_test_no_corr)

Wstępny model, do wykrycia ilości sąsiadów

In [ ]:
error_rate_test = []
for i in range(1, 20):
  knn = KNeighborsClassifier(n_neighbors= i, metric = 'euclidean', weights='uniform')
  knn.fit(x_train_std, y_train_array.ravel())
  pred_i = knn.predict(x_test_std).reshape((-1, 1))
  error_rate_test.append(np.mean(pred_i != y_test_array))

plt.rcParams['figure.figsize'] = [8, 5]
plt.plot(range(1, 20), error_rate_test, color ='red',
				linestyle ='dashed', marker ='o',
		markerfacecolor ='yellow', markersize = 10)

plt.title('Współczynnik błędu względem ilości sąsiadów')
plt.xlabel('Ilość sąsiadów')
plt.ylabel('Współczynnik błędu')
plt.show()

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 11, weights='uniform', metric='euclidean')
_ = knn.fit(x_train_std, y_train_array.ravel())

In [ ]:
y_predict_train_knn = knn.predict(x_train_std).reshape((-1, 1))

y_predict_test_knn = knn.predict(x_test_std).reshape((-1, 1))

conf_matrix_knn_train = confusion_matrix(y_train_array, y_predict_train_knn)
acc_knn_train_with_1 = np.diag(conf_matrix_knn_train, -1).sum() + np.diag(conf_matrix_knn_train).sum() + np.diag(conf_matrix_knn_train, 1).sum()
acc_knn_train_with_1 /= conf_matrix_knn_train.sum()

conf_matrix_knn_test = confusion_matrix(y_test_array, y_predict_test_knn)
acc_knn_test_with_1 = np.diag(conf_matrix_knn_test, -1).sum() + np.diag(conf_matrix_knn_test).sum() + np.diag(conf_matrix_knn_test, 1).sum()
acc_knn_test_with_1 /= conf_matrix_knn_test.sum()

In [ ]:
plt.rcParams['figure.figsize'] = [5, 5]
sns.heatmap(conf_matrix_knn_test, annot=True, fmt="d", linewidths=.5, xticklabels=[i for i in range(3, 9)], yticklabels=[i for i in range(3, 9)])
plt.show()

In [ ]:
print('MSE dla uczacej             ', mean_squared_error(y_train_array, y_predict_train_knn))
print('MAE dla uczacej             ', mean_absolute_error(y_train_array, y_predict_train_knn))
print('RMSE dla uczacej            ', np.sqrt(mean_squared_error(y_train_array, y_predict_train_knn)))
print('Trafność dla uczącej:       ', accuracy_score(y_train_array, y_predict_train_knn))
print('Trafność dla uczącej[+/-1]: ', acc_knn_train_with_1)
print('')
print('MSE dla testowej            ', mean_squared_error(y_test_array, y_predict_test_knn))
print('MAE dla testowej            ', mean_absolute_error(y_test_array, y_predict_test_knn))
print('RMSE dla testowej           ', np.sqrt(mean_squared_error(y_test_array, y_predict_test_knn)))
print('Trafność dla testowej:      ', accuracy_score(y_test_array, y_predict_test_knn))
print('Trafność dla testowej[+/-1]:', acc_knn_test_with_1)

In [ ]:
plt.rcParams['figure.figsize'] = [5, 5]
plt.scatter(x=y_test_array, y=y_predict_test_knn)
plt.xlabel('Wartości rzeczywiste')
plt.ylabel('Wartości przewidywane')
plt.show()

#Regresja -> Sieć neuronowa

In [ ]:
scaler_x = MinMaxScaler()
scaler_x.fit(x_train_no_corr)
x_train_norm = scaler_x.transform(x_train_no_corr)
x_test_norm = scaler_x.transform(x_test_no_corr)

scaler_y = MinMaxScaler()
scaler_y.fit(y_train_array)
y_train_norm = scaler_y.transform(y_train_array)
y_test_norm = scaler_y.transform(y_test_array)

In [ ]:
siec_neur = MLPRegressor(hidden_layer_sizes=(10,), activation='tanh', solver='lbfgs', alpha=0.0001, max_iter = 10000, random_state=index)
_ = siec_neur.fit(x_train_norm, y_train_norm.ravel())

In [ ]:
y_predict_train = siec_neur.predict(x_train_norm)
y_predict_train = y_predict_train.reshape((-1, 1))
y_predict_train_denorm = scaler_y.inverse_transform(y_predict_train)
y_predict_train_denorm_round = np.round(y_predict_train_denorm)

y_predict_test = siec_neur.predict(x_test_norm)
y_predict_test = y_predict_test.reshape((-1, 1))
y_predict_test_denorm = scaler_y.inverse_transform(y_predict_test)
y_predict_test_denorm_round = np.round(y_predict_test_denorm)

conf_matrix_siec_neur_train = confusion_matrix(y_train_array, y_predict_train_denorm_round)
acc_train_with_1 = np.diag(conf_matrix_siec_neur_train, -1).sum() + np.diag(conf_matrix_siec_neur_train).sum() + np.diag(conf_matrix_siec_neur_train, 1).sum()
acc_train_with_1 /= conf_matrix_siec_neur_train.sum()

conf_matrix_siec_neur_test = confusion_matrix(y_test_array, y_predict_test_denorm_round)
acc_test_with_1 = np.diag(conf_matrix_siec_neur_test, -1).sum() + np.diag(conf_matrix_siec_neur_test).sum() + np.diag(conf_matrix_siec_neur_test, 1).sum()
acc_test_with_1 /= conf_matrix_siec_neur_test.sum()

In [ ]:
plt.rcParams['figure.figsize'] = [5, 5]
sns.heatmap(conf_matrix_siec_neur_test, annot=True, fmt="d", linewidths=.5, xticklabels=[i for i in range(3, 9)], yticklabels=[i for i in range(3, 9)])
plt.show()

In [ ]:
print('MSE dla uczacej             ', mean_squared_error(y_train_array, y_predict_train_denorm_round))
print('MAE dla uczacej             ', mean_absolute_error(y_train_array, y_predict_train_denorm_round))
print('RMSE dla uczacej            ', np.sqrt(mean_squared_error(y_train_array, y_predict_train_denorm_round)))
print('Trafność dla uczącej:       ', accuracy_score(y_train_array, y_predict_train_denorm_round))
print('Trafność dla uczącej[+/-1]: ', acc_train_with_1)
print('')
print('MSE dla testowej            ', mean_squared_error(y_test_array, y_predict_test_denorm_round))
print('MAE dla testowej            ', mean_absolute_error(y_test_array, y_predict_test_denorm_round))
print('RMSE dla testowej           ', np.sqrt(mean_squared_error(y_test_array, y_predict_test_denorm_round)))
print('Trafność dla testowej:      ', accuracy_score(y_test_array, y_predict_test_denorm_round))
print('Trafność dla testowej[+/-1]:', acc_test_with_1)

In [ ]:
plt.close()
plt.rcParams['figure.figsize'] = [5, 5]
plt.scatter(x=y_test_array, y=y_predict_test_denorm_round)
plt.xlabel('Wartości rzeczywiste')
plt.ylabel('Wartości przewidywane')
plt.show()

#Grupowanie -> SOM

In [ ]:
dane_som_train = stats.zscore(wina_train[columns_train])
dane_som_test = stats.zscore(wina_test[columns_train])

In [ ]:
som = SOM(m = 4, n = 1, dim = len(columns_train))
np.random.seed(None)
np.random.seed(index)
som.fit(dane_som_train)

In [ ]:
wina_train['Klaster'] = som.predict(dane_som_train)
wina_test['Klaster'] = som.predict(dane_som_test)

In [ ]:
wina_train.groupby('Klaster').size().reset_index(name='Ilość')

In [ ]:
wina_train.groupby('Klaster').mean().reset_index().T

In [ ]:
wina_test.groupby('Klaster').size().reset_index(name='Ilość')

In [ ]:
wina_test.groupby('Klaster').mean().reset_index().T

In [ ]:
wina_srednie_train = wina_train.groupby('Klaster').mean()
wina_srednie_train['Zbiór'] = 'Treningowy'
wina_srednie_train['Ilość'] = wina_train.groupby('Klaster').size().reset_index(name='Ilość')['Ilość']

wina_srednie_test = wina_test.groupby('Klaster').mean()
wina_srednie_test['Zbiór'] = 'Testowy'
wina_srednie_test['Ilość'] = wina_test.groupby('Klaster').size().reset_index(name='Ilość')['Ilość']

In [ ]:
pd.concat([wina_srednie_train, wina_srednie_test]).sort_values(by=['Klaster']).T